In [1]:

import json
import pycamhd.lazycache as pycamhd

import paths
import camhd_motion_analysis as ma

mov_path = '/RS03ASHS/PN03B/06-CAMHDA301/2016/01/01/CAMHDA301-20160101T000000Z.mov'

metadata_repo = "/tmp"
data_filename = "/CAMHDA301-20160101T000000Z_optical_flow.json"

stride = 10

# Retrieve movie metadata

Get the number of frames in the movie

In [2]:
repo = pycamhd.lazycache()

movie_info = repo.get_metadata( url=mov_path )
print(movie_info)

{'URL': '/RS03ASHS/PN03B/06-CAMHDA301/2016/01/01/CAMHDA301-20160101T000000Z.mov', 'NumFrames': 25155, 'Duration': 839.33856}


## Single-threaded version

In [3]:
%%time

frames = range( 5000, 5010, stride )

results = [ma.frame_stats(mov_path, f) for f in frames]
print(results)

/home/aaron/workspace/camhd_analysis/camhd_motion_analysis/build-Debug/bin/frame_stats
[{'contents': {'frame_stats': {'contents': {'optical_flow': '1.0'}}, 'movie': '1.0', 'timing': '1.0'}, 'frame_stats': [{'frame_number': 5000, 'similarity': {'center': [-960.0, -540.0], 'duration_us': 11007677, 'flowScale': 1.0, 'fromFrame': 4995, 'imgScale': 0.25, 'similarity': [1.00015877249236, 0.0, 18.386438176448, 27.9921960533508], 'toFrame': 5005, 'valid': True}}], 'movie': {'Duration': 839.338562011719, 'NumFrames': 25155, 'URL': '/RS03ASHS/PN03B/06-CAMHDA301/2016/01/01/CAMHDA301-20160101T000000Z.mov', 'cacheURL': 'https://camhd-app-dev.appspot.com/v1/org/oceanobservatories/rawdata/files/RS03ASHS/PN03B/06-CAMHDA301/2016/01/01/CAMHDA301-20160101T000000Z.mov'}, 'timing': {'elapsed_system_time_s': 11.304482188}}]
CPU times: user 0 ns, sys: 12 ms, total: 12 ms
Wall time: 11.4 s


## Parallelize using Dask scheduler

In [ ]:
frames = range( 5000, 5020, stride )

from dask import compute, delayed
values = [delayed(frame_stats)(mov_path,f) for f in frames]


In [ ]:
%%time 

import dask.threaded
results = compute(*values, get=dask.threaded.get)

In [ ]:
## build a compliant JSON output


joutput = results[0]
for i in range(1, len(results)):
    joutput["frame_stats"].extend(results[i]["frame_stats"])

    
with open(metadata_repo + data_filename,'w') as f:
    json.dump(joutput, f, indent=2)